In [1]:
import os
import pathlib

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
# Установите seed для воспроизводимости
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
# Путь к вашему датасету
DATASET_PATH = '/home/redalexdad/recognition_speech/RUSLAN'

In [4]:
# Функция для декодирования аудио
def decode_audio(audio_binary):
    audio, _ = tf.audio.decode_wav(contents=audio_binary)
    return tf.squeeze(audio, axis=-1)

In [19]:
def get_label(file_path):
    # Преобразование float32 в строку, если это необходимо
    # Преобразование тензора в строку, если это необходимо
    if file_path.dtype != tf.string:
        file_path = tf.strings.as_string(file_path)

    # Разделение пути файла
    parts = tf.strings.split(input=file_path, sep=os.path.sep)

    # Получение метки из последней части пути
    label = parts[-2]

    return label

In [20]:
def get_waveform_and_label(file_path):
    # Преобразование float32 в строку, если это необходимо
    file_path = tf.cond(
        tf.math.equal(tf.rank(file_path), 0),
        lambda: tf.strings.as_string(file_path),
        lambda: tf.strings.as_string(tf.cast(file_path, tf.float32))  # Приведение к float32 перед преобразованием в строку
    )

    # Чтение аудиофайла
    audio_binary = tf.io.read_file(file_path)

    # Декодирование аудиофайла
    audio, _ = tf.audio.decode_wav(audio_binary)

    # Получение метки
    label = get_label(file_path)

    return audio, label

In [21]:
# Загрузка файлов и перемешивание
data_dir = pathlib.Path(DATASET_PATH)
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*.wav')
filenames = tf.random.shuffle(filenames)

In [22]:
# Разделение на тренировочный, валидационный и тестовый наборы
train_files = filenames[:6400]
val_files = filenames[6400: 6400 + 800]
test_files = filenames[-800:]

In [23]:
def get_spectrogram_and_label_id(audio, label):
    # Применяем преобразование Фурье для вычисления спектрограммы
    spectrogram = tf.signal.stft(
        audio, frame_length=255, frame_step=128)

    # Преобразуем комплексные числа в вещественные
    spectrogram = tf.abs(spectrogram)

    # Извлекаем метку идентификатора класса
    label_id = tf.argmax(label == commands)

    return spectrogram, label_id

In [24]:
# Функция для предобработки датасета
def preprocess_dataset(files):
    files_ds = tf.data.Dataset.from_tensor_slices(files)
    output_ds = files_ds.map(
        map_func=get_waveform_and_label,
        num_parallel_calls=tf.data.AUTOTUNE
    )
    output_ds = output_ds.map(
        map_func=get_spectrogram_and_label_id,
        num_parallel_calls=tf.data.AUTOTUNE
    )
    return output_ds

In [25]:
# Создание датасетов
train_ds = preprocess_dataset(train_files).batch(64).cache().prefetch(tf.data.AUTOTUNE)
val_ds = preprocess_dataset(val_files).batch(64).cache().prefetch(tf.data.AUTOTUNE)
test_ds = preprocess_dataset(test_files).batch(64).cache().prefetch(tf.data.AUTOTUNE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


NameError: in user code:

    /tmp/ipykernel_334422/4146273947.py:10 get_spectrogram_and_label_id  *
        label_id = tf.argmax(label == commands)

    NameError: name 'commands' is not defined


In [3]:

# Определение архитектуры модели
model = models.Sequential([
    layers.Input(shape=(None,)),  # Вход для волновой формы аудио
    layers.Reshape((-1, 1)),  # Преобразование в трехмерный тензор для свертки
    layers.Conv1D(32, 3, activation='relu'),
    layers.MaxPooling1D(),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_labels),
])

NameError: name 'models' is not defined

In [ ]:
# Компиляция модели
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
# Обучение модели
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)